In [3]:
from spatialSSL.Dataloader import FullImageDatasetConstructor
from spatialSSL.Utils import split_dataset
from spatialSSL.Training import train, train_classification
from spatialSSL.Models import *
from spatialSSL.Training import train_epoch
from spatialSSL.Testing import test
from spatialSSL.Dataset import InMemoryGraphDataset
import numpy as np

C:\Users\lg\anaconda3\envs\spatial_atlas_ssl\lib\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\lg\anaconda3\envs\spatial_atlas_ssl\lib\site-packages\spatialdata\__init__.py:9: UserWarning: Geopandas was set to use PyGEOS, changing to shapely 2.0 with:

	geopandas.options.use_pygeos = True

If you intended to use PyGEOS, set the option to False.
  _check_geopandas_using_shapely()


In [4]:
import torch
import zipfile

# Define a function to load the data from the ZIP file
def load_from_zip(zip_path, file_name):
    with zipfile.ZipFile(zip_path, 'r') as zipf:
        with zipf.open(file_name) as file:
            return torch.load(file)

# Load the pre_train_list and pre_val_list from the ZIP file
pre_train_list = load_from_zip('./processed_data/pre_training_data_img6_r30_n1_random_01.zip', 'pre_train_list.pt')
pre_val_list = load_from_zip('./processed_data/pre_training_data_img6_r30_n1_random_01.zip', 'pre_val_list.pt')

# Now pre_train_list and pre_val_list contain the loaded data


In [5]:
from torch_geometric.loader import DataLoader

# Create DataLoader objects for pre-training and pre-validation
pre_train_loader = DataLoader(pre_train_list, batch_size=1, shuffle=True)
pre_val_loader = DataLoader(pre_val_list, batch_size=1, shuffle=False)

# Now you can use pre_train_loader and pre_val_loader in your training loop


In [ ]:
device

In [ ]:
# Run pretraining

in_channels  = 550
hidden_channels_1 = 256
hidden_channels_2 = 33
out_channels= 550

# Pretraining
# Define the device
device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu') #"cpu"

# Create the model
model = GAT4(in_channels, hidden_channels_1,hidden_channels_2, out_channels).to(device) # in_channels is set to 100 as an example. Please replace it with your actual feature size.

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 300
patience = 8

train(model=model, train_loader=pre_train_loader, val_loader=pre_val_loader, criterion=criterion, num_epochs= num_epochs, patience = patience, optimizer= optimizer,weight_loss = False, model_path="./models/img6_r30_n1_random_01_GAT4_0.001.pt'")

In [6]:
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss

# Run pretraining

in_channels  = 550
hidden_channels = 256
num_classes= 33

# Pretraining
# Define the device
device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu') #"cpu"

model = GATClassification(in_channels, hidden_channels, num_classes).to(device)
criterion = CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_classification(model, pre_train_loader, pre_val_loader, criterion, num_classes=num_classes,optimizer=optimizer)

tensor([[ 6.0140,  1.6878,  2.3021,  ..., -4.1208, -3.9464, -3.9208],
        [ 5.8610,  3.2718,  2.7216,  ..., -4.3936, -4.1666, -4.2311],
        [ 5.2924,  2.2868,  2.2859,  ..., -3.9851, -3.6019, -3.9323],
        ...,
        [ 6.6221,  2.8121,  2.8406,  ..., -4.6768, -4.4690, -4.7496],
        [ 5.6934,  1.9169,  2.0258,  ..., -3.8681, -4.0155, -4.0442],
        [ 5.2665,  9.9723,  7.2238,  ..., -5.1825, -4.4036, -4.9247]])
tensor([19, 19, 17,  ..., 23, 16,  2])
tensor([[ 6.9195,  3.3384,  2.8896,  ..., -4.3925, -4.2627, -4.4159],
        [ 4.0869,  1.0489,  1.5870,  ..., -3.0403, -2.9295, -2.8694],
        [ 4.5162,  9.8908,  4.9477,  ..., -4.6207, -3.8057, -4.3769],
        ...,
        [ 5.7730, 10.2075,  5.9910,  ..., -5.1711, -4.2671, -4.8393],
        [ 5.4036,  2.4759,  2.2818,  ..., -3.9024, -3.8399, -4.0565],
        [ 4.6939,  1.4457,  1.7119,  ..., -3.2026, -3.0734, -3.0253]])
tensor([19, 19,  1,  ...,  1,  0, 23])
Epoch 2/100, train loss: 1.3542, train accuracy: 0.437

KeyboardInterrupt: 